# **EDA and Hypothesis Testing of Ecological Variables**

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, kruskal, f_oneway
from scikit_posthocs import posthoc_dunn
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data (cleaning and scaling)
df = pd.read_csv('AVONETplusClim.csv')
df_clean = df.copy()
df_clean = df_clean[(df_clean['Mass'] > 0) & (df_clean['Mass'] < 11500)]
df_clean = df_clean[(df_clean['Tarsus.Length'] > 0.1) & (df_clean['Tarsus.Length'] < 535)]
df_clean = df_clean[(df_clean['Tail.Length'] > 0) & (df_clean['Tail.Length'] < 350)]
df_clean = df_clean[(df_clean['Wing.Length'] > 0.1) & (df_clean['Wing.Length'] < 650)]
df_clean = df_clean[df_clean['Hand-Wing.Index'] > 3]
df_clean['log_Mass'] = np.log10(df_clean['Mass'])
df_clean['log_Tarsus'] = np.log10(df_clean['Tarsus.Length'])
df_clean['log_Tail'] = np.log10(df_clean['Tail.Length'])
df_clean['log_Beak'] = np.log10(df_clean['Beak.Length_Culmen'])
df_clean['log_Wing'] = np.log10(df_clean['Wing.Length'])
df_clean['log_HWI'] = np.log10(df_clean['Hand-Wing.Index'])

## **1. Body Size (Mass)**

### **1.1. Mass and Habitat**

Before starting, distribution of mass within each habitat group is explored based on skewness and kurtosis. Since they are numerous, habitat types are classified into three categories (open, closed, aquatic). 

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_Mass']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(Mass) (log(g))", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Mass by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The closed habitat group could be assumed to show a normal distribution of mass. However, the open hapitat group has a high skew (skewness = 1) and the aquatic habitat group shows a flat distribution (kurtosis < -1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis. 

* H0: The distribution of mass is the same among habitat groups.
* H1: The distribution of mass significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Mass'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of mass among at least two habitat groups. To see which habitat groups differ, Dunn's test is done, in which

* H0: The distribution of mass is the same for the two habitat groups tested.
* H1: The distribution of mass differs between the two habitat groups tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Mass', 
                                group_col='Habitat_Group', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in mass distribution between every single pair of habitat groups. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_Mass',
    title='<b>Distribution of Mass by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(Mass) (log(g))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **1.2. Mass and Migration**

Before starting, distribution of mass within each migration behavior is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Sedentary', 'Partial', 'Migratory')
)

configs = [
    ('Sedentary', 'saddlebrown', 1),
    ('Partial', 'goldenrod', 2),
    ('Migratory', 'skyblue', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Migration'] == label]['log_Mass']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(Mass) (log(g))", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of Mass by Migration</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The sedentary and partial classes could be assumed to show a normal distribution of mass. However, the migratory class has a high skew (skewness > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of mass is the same among migratory classes.
* H1: The distribution of mass significantly differ among at least two migratory classes.

In [ ]:
groups = sorted(df_clean['Migration'].unique())
data_groups = [df_clean[df_clean['Migration'] == g]['Mass'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference of mass among at least two migratory classes. To see which migratory classess differ, Dunn's test is done, in which

* H0: The distribution of mass is the same for the two migratory classes tested.
* H1: The distribution of mass differs between the two migratory classes tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Mass', 
                                group_col='Migration', 
                                p_adjust='bonferroni')

print(dunn_results)

Between sedentary and migratory classes, p > 0.05, thus the null hypothesis (H0) is failed to reject. However, for the pairs partial-sedentary and partial-migratory, p << 0.05, thus there are statistically significant differences in mass distribution between these migratory classes. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Migration'].isin(['Sedentary', 'Partial', 'Migratory'])].copy()

fig = px.box(
    df_plot, 
    x='Migration', 
    y='log_Mass',
    title='<b>Distribution of Mass by Migration</b>',
    color='Migration',
    color_discrete_sequence=['saddlebrown', 'goldenrod', 'skyblue'],
    category_orders={"Migration": ['Sedentary', 'Partial', 'Migratory']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Migration Behavior',
    yaxis_title='log(Mass) (log(g))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **1.3. Mass and Diet (Trophic Level)**

Before starting, distribution of mass within each trophic level is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Carnivore', 'Herbivore', 'Omnivore')
)

configs = [
    ('Carnivore', 'firebrick', 1),
    ('Herbivore', 'forestgreen', 2),
    ('Omnivore', 'goldenrod', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Trophic.Level'] == label]['log_Mass']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(Mass) (log(g))", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of Mass by Trophic Level</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The herbivores could be assumed to show a normal distribution of mass. However, the carnivores and omnivores have a high skew (skewness > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of mass is the same among trophic levels.
* H1: The distribution of mass significantly differ among at least two trophic levels.

In [ ]:
df_trophic = df_clean[df_clean['Trophic.Level'].isin(['Carnivore', 'Herbivore', 'Omnivore'])]
data_groups = [df_trophic[df_trophic['Trophic.Level'] == g]['Mass'] for g in ['Carnivore', 'Herbivore', 'Omnivore']]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of mass among at least two trophic levels. To see which trophic levels differ, Dunn's test is done, in which

* H0: The distribution of mass is the same for the two trophic levels tested.
* H1: The distribution of mass differs between the two trophic levels tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_trophic, 
                                val_col='Mass', 
                                group_col='Trophic.Level', 
                                p_adjust='bonferroni')

print(dunn_results)

Between herbivores and omnivores, p > 0.05, thus the null hypothesis is failed to reject. However, the remaining pairs show statistically significant differences (p < 0.05) in mass distribution. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
fig = px.box(
    df_trophic, 
    x='Trophic.Level', 
    y='log_Mass',
    title='<b>Distribution of Mass by Trophic Level</b>',
    color='Trophic.Level',
    color_discrete_sequence=['firebrick', 'forestgreen', 'goldenrod'],
    category_orders={"Trophic.Level": ['Carnivore', 'Herbivore', 'Omnivore']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Trophic Level',
    yaxis_title='log(Mass) (log(g))',
    template='plotly_white',
    showlegend=False
)

fig.show()

## **2. Leg Size (Tarsus Length)**

### **2.1. Tarsus Length and Habitat**

Before starting, distribution of tarsus length within each habitat group is explored based on skewness and kurtosis.

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_Tarsus']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Tarsus Length by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The aquatic habitat group could be assumed to show a normal distribution of tarsus length. However, the closed and open habitat groups have high peaks (kurtosis > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of tarsus length is the same among habitat groups.
* H1: The distribution of tarsus length significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Tarsus.Length'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of tarsus length among at least two habitat groups. To see which habitat groups differ, Dunn's test is done, in which

* H0: The distribution of tarsus length is the same for the two habitat groups tested.
* H1: The distribution of tarsus length differs between the two habitat groups tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Tarsus.Length', 
                                group_col='Habitat_Group', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in tarsus length between every single pair of habitat groups. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_Tarsus',
    title='<b>Distribution of Tarsus Length by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

## **3. Tail Size (Tail Length)**

### **3.1. Tail Length and Habitat**

Before starting, distribution of tail length within each habitat group is explored based on skewness and kurtosis.

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_Tail']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Tail Length by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

All the habitat groups have moderate skew (-1 < skewness < 1) and moderate peak (-1 < kurtosis < 1) and therefore can be assumed to show a normal distribution of tail length. It is safe to use a parametric test, which is ANOVA.

* H0: The distribution of tail length is the same among habitat groups.
* H1: The distribution of tail length significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Tail.Length'] for g in groups]

f_stat, p_val = f_oneway(*data_groups)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of tail length among at least two habitat groups. To see which habitat groups differ, Tukey's test is done, in which

* H0: The distribution of tail length is the same for the two habitat groups tested.
* H1: The distribution of tail length differs between the two habitat groups tested.

In [ ]:
tukey = pairwise_tukeyhsd(
    endog=df_clean['Tail.Length'],
    groups=df_clean['Habitat_Group'],
    alpha=0.05
)

print(tukey)

Between aquatic and open habitats, p > 0.05, thus the null hypothesis (H0) is failed to reject. However, the remaining pairs show statistically significant differences (p < 0.05) in tail length distribution. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_Tail',
    title='<b>Distribution of Tail Length by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **3.2. Tail Length and Migration**

Before starting, distribution of tail length within each migration behavior is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Sedentary', 'Partial', 'Migratory')
)

configs = [
    ('Sedentary', 'saddlebrown', 1),
    ('Partial', 'goldenrod', 2),
    ('Migratory', 'skyblue', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Migration'] == label]['log_Tail']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of Tail Length by Migration</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

All the migratory classes have moderate skew (-1 < skewness < 1) and moderate peak (-1 < kurtosis < 1) and therefore can be assumed to show a normal distribution of tail length. It is safe to use a parametric test, which is ANOVA.

* H0: The distribution of tail length is the same among migratory classes.
* H1: The distribution of tail length significantly differ among at least two migratory classes.

In [ ]:
groups = sorted(df_clean['Migration'].unique())
data_groups = [df_clean[df_clean['Migration'] == g]['Tail.Length'] for g in groups]

f_stat, p_val = f_oneway(*data_groups)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of tarsus length among at least two migratory classes. To see which migratory classess differ, Tukey's test is done, in which

* H0: The distribution of tarsus length is the same for the two migratory classes tested.
* H1: The distribution of tarsus length differs between the two migratory classes tested.

In [ ]:
tukey = pairwise_tukeyhsd(
    endog=df_clean['Tail.Length'],
    groups=df_clean['Migration'],
    alpha=0.05
)

print(tukey)

Between migratory and sedentary behaviors, p > 0.05, thus the null hypothesis (H0) is failed to reject. However, for the remaining pairs show statistically significant differences (p < 0.05) in tail length distribution. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Migration'].isin(['Sedentary', 'Partial', 'Migratory'])].copy()

fig = px.box(
    df_plot, 
    x='Migration', 
    y='log_Tail',
    title='<b>Distribution of Tail Length by Migration</b>',
    color='Migration',
    color_discrete_sequence=['saddlebrown', 'goldenrod', 'skyblue'],
    category_orders={"Migration": ['Sedentary', 'Partial', 'Migratory']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Migration Behavior',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

## **4. Beak Size (Beak Length)**

### **4.1. Beak Length and Habitat**

Before starting, distribution of beak length within each habitat group is explored based on skewness and kurtosis.

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_Beak']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Beak Length by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The aquatic habitat group could be assumed to show a normal distribution of tarsus length. However, the closed and open habitat groups have high peaks (kurtosis > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of beak length is the same among habitat groups.
* H1: The distribution of beak length significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Beak.Length_Culmen'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of beak length among at least two habitat groups. To see which habitat groups differ, Dunn's test is done, in which

* H0: The distribution of beak length is the same for the two habitat groups tested.
* H1: The distribution of beak length differs between the two habitat groups tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Beak.Length_Culmen', 
                                group_col='Habitat_Group', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in beak length between every single pair of habitat groups. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_Beak',
    title='<b>Distribution of Beak Length by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **4.2. Beak Length and Diet (Trophic Level)**

Before starting, distribution of beak length within each trophic level is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Carnivore', 'Herbivore', 'Omnivore')
)

configs = [
    ('Carnivore', 'firebrick', 1),
    ('Herbivore', 'forestgreen', 2),
    ('Omnivore', 'goldenrod', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Trophic.Level'] == label]['log_Beak']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of Beak Length by Trophic Level</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

All the trophic levels have high peaks (kurtosis > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of beak length is the same among trophic levels.
* H1: The distribution of beak length significantly differ among at least two trophic levels.

In [ ]:
df_trophic = df_clean[df_clean['Trophic.Level'].isin(['Carnivore', 'Herbivore', 'Omnivore'])]
data_groups = [df_trophic[df_trophic['Trophic.Level'] == g]['Beak.Length_Culmen'] for g in ['Carnivore', 'Herbivore', 'Omnivore']]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p < 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of beak length among at least two trophic levels. To see which trophic levels differ, Dunn's test is done, in which

* H0: The distribution of beak length is the same for the two trophic levels tested.
* H1: The distribution of beak length differs between the two trophic levels tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_trophic, 
                                val_col='Tarsus.Length', 
                                group_col='Trophic.Level', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in beak length between every single pair of trophic levels. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
fig = px.box(
    df_trophic, 
    x='Trophic.Level', 
    y='log_Beak',
    title='<b>Distribution of Beak Length by Trophic Level</b>',
    color='Trophic.Level',
    color_discrete_sequence=['firebrick', 'forestgreen', 'goldenrod'],
    category_orders={"Trophic.Level": ['Carnivore', 'Herbivore', 'Omnivore']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Trophic Level',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

## **5. Wing Size and Shape (Wing Length and Hand-Wing Index)**

### **5.1. Wing Length and Habitat**

Before starting, distribution of wing length within each habitat group is explored based on skewness and kurtosis.

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_Wing']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Wing Length by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

All the habitat groups have moderate skew (-1 < skewness < 1) and moderate peak (-1 < kurtosis < 1) and therefore can be assumed to show a normal distribution of wing length. It is safe to use a parametric test, which is ANOVA.

* H0: The distribution of wing length is the same among habitat groups.
* H1: The distribution of wing length significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Wing.Length'] for g in groups]

f_stat, p_val = f_oneway(*data_groups)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of wing length among at least two habitat groups. To see which habitat groups differ, Tukey's test is done, in which

* H0: The distribution of wing length is the same for the two habitat groups tested.
* H1: The distribution of wing length differs between the two habitat groups tested.

In [ ]:
tukey = pairwise_tukeyhsd(
    endog=df_clean['Wing.Length'],
    groups=df_clean['Habitat_Group'],
    alpha=0.05
)

print(tukey)

All the pairs show statistically significant differences (p < 0.05) in wing length distribution. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_Wing',
    title='<b>Distribution of Wing Length by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **5.2. Wing Length and Migration**

Before starting, distribution of wing length within each migratory behavior is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Sedentary', 'Partial', 'Migratory')
)

configs = [
    ('Sedentary', 'saddlebrown', 1),
    ('Partial', 'goldenrod', 2),
    ('Migratory', 'skyblue', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Migration'] == label]['log_Wing']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(Length) (log(mm))", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of Wing Length by Migration</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

All the migratory classes have moderate skew (-1 < skewness < 1) and moderate peak (-1 < kurtosis < 1) and therefore can be assumed to show a normal distribution of wing length. It is safe to use a parametric test, which is ANOVA.

* H0: The distribution of wing length is the same among migratory classes.
* H1: The distribution of wing length significantly differ among at least two migratory classes.

In [ ]:
groups = sorted(df_clean['Migration'].unique())
data_groups = [df_clean[df_clean['Migration'] == g]['Wing.Length'] for g in groups]

f_stat, p_val = f_oneway(*data_groups)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of wing length among at least two migratory classes. To see which migratory classess differ, Tukey's test is done, in which

* H0: The distribution of wing length is the same for the two migratory classes tested.
* H1: The distribution of wing length differs between the two migratory classes tested.

In [ ]:
tukey = pairwise_tukeyhsd(
    endog=df_clean['Wing.Length'],
    groups=df_clean['Migration'],
    alpha=0.05
)

print(tukey)

All the pairs show statistically significant differences (p < 0.05) in wing length distribution. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Migration'].isin(['Sedentary', 'Partial', 'Migratory'])].copy()

fig = px.box(
    df_plot, 
    x='Migration', 
    y='log_Wing',
    title='<b>Distribution of Wing Length by Migration</b>',
    color='Migration',
    color_discrete_sequence=['saddlebrown', 'goldenrod', 'skyblue'],
    category_orders={"Migration": ['Sedentary', 'Partial', 'Migratory']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Migration Behavior',
    yaxis_title='log(Length) (log(mm))',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **5.3. Hand-Wing Index (HWI) and Habitat**

Before starting, distribution of HWI within each habitat group is explored based on skewness and kurtosis.

In [ ]:
conditions = [
    # Aquatic/Marine
    df_clean["Habitat"].isin(['Coastal', 'Marine', 'Riverine', 'Wetland']),
    # Closed/Forest
    df_clean["Habitat"].isin(['Forest', 'Woodland', 'Shrubland']),
    # Open/Terrestrial
    df_clean["Habitat"].isin(['Grassland', 'Desert', 'Rock', 'Human Modified'])
]
choices = ['Aquatic', 'Closed', 'Open']
df_clean["Habitat_Group"] = np.select(conditions, choices, default=None)

########################################################################

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("Closed Habitat", "Open Habitat", "Aquatic Habitat")
)

plot_configs = [
    ('Closed', 'forestgreen', 1),
    ('Open', 'sandybrown', 2),
    ('Aquatic', 'deepskyblue', 3)
]

for group_name, color, col_idx in plot_configs:

    subset = df_clean[df_clean['Habitat_Group'] == group_name]['log_HWI']
    s, k = skew(subset), kurtosis(subset)
    fig.add_trace(
        go.Histogram(x=subset, nbinsx=50, name=group_name, marker_color=color),
        row=1, col=col_idx
    )
    
    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )
    
    fig.update_xaxes(title_text="log(HWI)", row=1, col=col_idx)

fig.update_layout(
    title_text="<b>Distribution of Hand-Wing Index by Habitat Group</b>",
    title_x=0.5,
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The the closed and open habitat groups could be assumed to show a normal distribution of HWI. However, aquatic habitat group has high peaks (kurtosis > 1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of HWI is the same among habitat groups.
* H1: The distribution of HWI significantly differ among at least two habitat groups.

In [ ]:
groups = sorted(df_clean['Habitat_Group'].unique())
data_groups = [df_clean[df_clean['Habitat_Group'] == g]['Hand-Wing.Index'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of HWI among at least two habitat groups. To see which habitat groups differ, Dunn's test is done, in which

* H0: The distribution of HWI is the same for the two habitat groups tested.
* H1: The distribution of HWI differs between the two habitat groups tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Hand-Wing.Index', 
                                group_col='Habitat_Group', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in HWI between every single pair of habitat groups. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Habitat_Group'].isin(['Closed','Open', 'Aquatic'])].copy()

fig = px.box(
    df_plot, 
    x='Habitat_Group', 
    y='log_HWI',
    title='<b>Distribution of HWI by Habitat Group</b>',
    color='Habitat_Group',
    color_discrete_sequence=['forestgreen', 'sandybrown', 'deepskyblue'],
    category_orders={"Habitat": ['Closed', 'Open', 'Aquatic']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Habitat Group',
    yaxis_title='log(HWI)',
    template='plotly_white',
    showlegend=False
)

fig.show()

### **5.4. Hand-Wing Index (HWI) and Migration**

Before starting, distribution of HWI within each migration behavior is explored based on skewness and kurtosis.

In [ ]:
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('Sedentary', 'Partial', 'Migratory')
)

configs = [
    ('Sedentary', 'saddlebrown', 1),
    ('Partial', 'goldenrod', 2),
    ('Migratory', 'skyblue', 3)
]

for label, color, col_idx in configs:
    d_subset = df_clean[df_clean['Migration'] == label]['log_HWI']
    s, k = skew(d_subset), kurtosis(d_subset)

    fig.add_trace(
        go.Histogram(x=d_subset, nbinsx=50, name=label, marker_color=color), 
        row=1, col=col_idx
    )

    fig.add_annotation(
        text=f"Skewness: {s:.2f}<br>Kurtosis: {k:.2f}",
        xref="x domain", yref="y domain",
        x=1, y=0.95, showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="black", borderwidth=1,
        row=1, col=col_idx
    )

    fig.update_xaxes(title_text="log(HWI)", row=1, col=col_idx)

fig.update_layout(
    title_x=0.5,
    title_text="<b>Distribution of HWI by Migration</b>",
    showlegend=False,
    bargap=0.1,
    template="plotly_white"
)

fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

The sedentary and partial classes could be assumed to show a normal distribution of HWI. However, the migratory class has a flat distribution (kurtosis < -1). Therefore, using a non-parametric test is is the safer option, which is Kruskal-Wallis.

* H0: The distribution of HWI is the same among migratory classes.
* H1: The distribution of HWI significantly differ among at least two migratory classes.

In [ ]:
groups = sorted(df_clean['Migration'].unique())
data_groups = [df_clean[df_clean['Migration'] == g]['Hand-Wing.Index'] for g in groups]

h_stat, p_val = kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of HWI among at least two migratory classes. To see which migratory classes differ, Dunn's test is done, in which

* H0: The distribution of HWI is the same for the two migratory classes tested.
* H1: The distribution of HWI differs between the two migratory classes tested.

In [ ]:
dunn_results = posthoc_dunn(a=df_clean, 
                                val_col='Hand-Wing.Index', 
                                group_col='Migration', 
                                p_adjust='bonferroni')

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in tarsus length between every single pair of migratory classes. Finally, to illustrate the distribution, a boxplot is created.

In [ ]:
df_plot = df_clean[df_clean['Migration'].isin(['Sedentary', 'Partial', 'Migratory'])].copy()

fig = px.box(
    df_plot, 
    x='Migration', 
    y='log_HWI',
    title='<b>Distribution of Hand-Wing Index by Migration</b>',
    color='Migration',
    color_discrete_sequence=['saddlebrown', 'goldenrod', 'skyblue'],
    category_orders={"Migration": ['Sedentary', 'Partial', 'Migratory']}
)

fig.update_layout(
    title_x=0.5,
    xaxis_title='Migration Behavior',
    yaxis_title='log(HWI)',
    template='plotly_white',
    showlegend=False
)

fig.show()